In [35]:
pip install tensorflow


In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import BatchNormalization
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score,
    average_precision_score, cohen_kappa_score, matthews_corrcoef
)

In [74]:
def scaling(X):
  scaler = StandardScaler()
  X= scaler.fit_transform(X)

  return X

In [75]:
def knn_imputer(data,original_column_names):
  data = pd.DataFrame(data, columns=original_column_names)
  knn_imputer = KNNImputer(n_neighbors=5)
  numeric_cols = data.select_dtypes(include=['number']).columns
  data[numeric_cols] = knn_imputer.fit_transform(data[numeric_cols])
  return data

In [76]:
def lin_data_prep(X,l_feature_names):
  X['numberofpop'] = np.log1p(X['numberofpop'])
  X=X.drop(columns=['pol_fact','amount_of_pol'])
  X['totaltrees'] = np.log1p(X['trees'])
  X['num_fact'] = (X['num_fact'] > 0).astype(int)


  return X

In [77]:
def run_neural_network_classifier(X_train, X_test, y_train, y_test, num_classes):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')  # פלט ל־n קטגוריות
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.005),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )


    y_pred = model.predict(X_test)
    y_pred_classes = y_pred.argmax(y_pred, axis=1)
    y_true_classes = y_test(y_test, axis=1)

  #  y_true_class = y_test.argmax(axis=1)



    print_metrics(y_true_classes, y_pred_classes, "Neural Network Classifier")
    print(classification_report(y_true_classes, y_pred_classes))

In [78]:
file_path ="/content/newdatajsonfortest.csv"

    # קריאת הקובץ
df_n = pd.read_csv(file_path)


In [70]:
"""def categorize_aqi(value):
if value >= 80:
        return 'Excellent'
    elif value >= 60:
        return 'Good'
    elif value >= 40:
        return 'Moderate'
    elif value >= 20:
        return 'Low'
    elif value >= 1:
        return 'Poor'
    else:
        return 'Very Poor'"""

"def categorize_aqi(value):\nif value >= 80:\n        return 'Excellent'\n    elif value >= 60:\n        return 'Good'\n    elif value >= 40:\n        return 'Moderate'\n    elif value >= 20:\n        return 'Low'\n    elif value >= 1:\n        return 'Poor'\n    else:\n        return 'Very Poor'"

In [79]:
def new_categorize_aqi(value):
    if value >= 68:
        return 'Exceptional'
    elif value >= 65:
        return 'Excellent'
    elif value >= 63:
        return 'Good'
    elif value >= 60:
        return 'Moderate'
    elif value >= 55:
        return 'Low'
    elif value >= 50:
        return 'Poor'
    else:
        return 'Very Poor'

In [82]:

target_column = "aqi_category"

df_n['aqi_category'] = df_n['aqi_global'].apply(new_categorize_aqi)

# y ו-X
y = df_n[target_column]
X = df_n.drop(columns=['hebname','lat','lan','aqi_global','aqi_category','belogns','type'])

feature_names =X.columns.tolist()
# ניקוי/הנדסת פיצ'רים
X = X.replace([np.inf, -np.inf], np.nan)
X = scaling(X)
X = knn_imputer(X, feature_names)

X["cars_per_person"] = X["cars"] / (X["numberofpop"] + 1e-6) * 1000
X["bus_per_person"]  = X["busStations"] / (X["numberofpop"] + 1e-6) * 1000
X = X.drop(columns=['cars','busStations'])

# חלוקה ל-Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# קידוד תוויות
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc  = le.transform(y_test)
num_classes = len(le.classes_)
print("Label map:", dict(zip(le.classes_, range(num_classes))))


model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train_enc,
          epochs=30, batch_size=32,
          validation_data=(X_test, y_test_enc),
          verbose=0)

# חיזוי והערכה
y_pred_proba  = model.predict(X_test)
y_pred        = np.argmax(y_pred_proba, axis=1)
print(classification_report(y_test_enc, y_pred, target_names=le.classes_))


Label map: {'Excellent': 0, 'Exceptional': 1, 'Good': 2, 'Low': 3, 'Moderate': 4, 'Poor': 5, 'Very Poor': 6}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   numberofpop      1000 non-null   float64
 1   city_heights     1000 non-null   float64
 2   num_fact         1000 non-null   float64
 3   pol_fact         1000 non-null   float64
 4   amount_of_pol    1000 non-null   float64
 5   trees            1000 non-null   float64
 6   cars_per_person  1000 non-null   float64
 7   bus_per_person   1000 non-null   float64
dtypes: float64(8)
memory usage: 62.6 KB


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
              precision    recall  f1-score   support

   Excellent       0.27      0.24      0.25        17
 Exceptional       0.19      0.07      0.10        45
        Good       0.00      0.00      0.00        13
         Low       0.40      0.08      0.13        52
    Moderate       0.31      0.88      0.46        57
        Poor       0.00      0.00      0.00        15
   Very Poor       0.00      0.00      0.00         1

    accuracy                           0.30       200
   macro avg       0.17      0.18      0.13       200
weighted avg       0.26      0.30      0.21       200



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [58]:
cm = confusion_matrix(y_test_enc, y_pred, normalize='true')
print(cm)

[[0.05882353 0.17647059 0.         0.29411765 0.47058824 0.
  0.        ]
 [0.04444444 0.2        0.         0.22222222 0.53333333 0.
  0.        ]
 [0.07692308 0.23076923 0.         0.23076923 0.38461538 0.07692308
  0.        ]
 [0.03846154 0.09615385 0.         0.30769231 0.55769231 0.
  0.        ]
 [0.03508772 0.0877193  0.         0.19298246 0.68421053 0.
  0.        ]
 [0.         0.         0.         0.6        0.4        0.
  0.        ]
 [0.         0.         0.         0.         1.         0.
  0.        ]]
